# Analysis of incomplete time series - Part 2

This part will focus on the analysis of irregular time series.

## Example 1: Reconstruction of cyanobacterial blooms

We read in a time series of a biomarker indicating past cyanobacterial blooms:

In [ ]:
Cyano_rec <- read.csv("../Data/Biomarker_Faeroe.csv",dec=',',header=TRUE,sep="\t")
head(Cyano_rec)
plot(Cyano_rec)

It seems like there is a positive trend: We have more cyanobacteria today. But can we prove it?

Let us first try a simple linear model. This assumes that the data show a linear rise plus a random error at each data point. The random errors are independent and normally distributed.

In [ ]:
# Fit the linear model
my_lm = lm(formula = Cya~Age,data=Cyano_rec)
# print a summary of the model fit
summary(my_lm)
# plot the data and the fitted model
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_lm,Cyano_rec))
# print the uncertainty interval for the slope
confint(my_lm)

The model finds a significantly positive trend of 0.0059 year$^{-1}$. But is the model suitable for the data? 
To find that out, we should **always have a look at the residuals**.

In [ ]:
plot(resid(my_lm))
hist(resid(my_lm))

The residuals violate both assumptions we made: 
* They are not independent: we see clustering especially of negative residuals.
* They are not normally distributed: We see that their distribution is skewed.
So, the model is inappropriate, **it is wrong to use its results**.

We can plot the autocorrelation between the residuals:

In [ ]:
acf(resid(my_lm))

We see that lag-1 and lag-2 autocorrelation are significantly positive. 
**If we ignore this, the model may dramatically underestimate the uncertainty of the slope!**

Next attempt: We assume that the residuals are autocorrelated in time. For irregular time series, we can use the continuous AR-1 autocorrelation and do a generalised least-squares fit:
$$E(\varepsilon_n \cdot \varepsilon_{n+1}) = \sigma^2 \exp(-r(t_{n+1}-t_n))$$

In [ ]:
library("nlme")
# do generalised least-squares fit
my_gls = gls(Cya~Age,data=Cyano_rec,correlation = corCAR1(form=~Age))
summary(my_gls)
# plot the data and the fitted model
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_gls,Cyano_rec))
# print the uncertainty interval for the slope
confint(my_gls)

Let's have a look at the new residuals:

In [ ]:
# Three types of residuals:
# "response"     = the full residuals epsilon_n
# "standardized" = divided by standard deviation sigma
# "normalized"   = divided by standard deviation sigma and corrected for assumed autocorrelation
my_residuals = resid(my_gls,type="normalized")

plot(my_residuals)
hist(my_residuals)
acf(my_residuals)

This is a model we could publish if we are interested in a linear trend. 

But fitting a straight line is not optimal for matching the data. Let's look at the plot again:


In [ ]:
# plot the data and the fitted model
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_gls,Cyano_rec))

Doesn't it look like there are basically two shifts: A rise in the 1920s to values around 0.7, and then another rise in the 1990s?

We can actually find this out by using a "nonparametric model".

## Nonparametric models

In classical regression models, we prescribe the shape of the trend (linear, quadratic, ...) and let the model estimate the coefficients only. 

In "nonparametric models", the model shall also estimate the shape of the response. A good idea is to use "splines".

## Splines

Splines (=smooth polinomial lines) are a nice way to fit smooth lines through given data.

Assume we have some points which we want to fit the data through:

In [ ]:
x_values = c(0,1,2,3)
y_values = c(1,2,4,1)
plot(x_values,y_values)

We could of course connect neighbouring points by straight lines, but that would not be smooth.
The slope would change abruptly at these "knots".

An alternative is to define the slope at the knots as well and then to fit a third-order polynomial between every two knots. 
A third-order polynomial
$$ y = ax^3 + bx^2 + cx + d $$
has four degrees of freedom, so it is uniquely defined by two y-values and two slopes at each end.

the R function __splinefunH()__ returns a function that fits the given knots and slopes:


In [ ]:
slopes = c(-1,2,0,-3)
my_splinefunction = splinefunH(x=x_values, y=y_values, m=slopes)  # determine spline function
x_axis = seq(from=0,to=3,by=0.1)                                  # define an x axis vector
my_spline = my_splinefunction(x_axis)                             # apply the function to the x values to obtain y values

# now plot the spline as line and the knots as points
plot(x_axis,my_spline,type="l")
points(x_values,y_values,col="red",pch=15)

This type of spline with cubic polynomials between the knots is called "Hermite spline". 

Assume we have some data which are scattered in $x$ and $y$. A non-parametric model will automatically optimise the vertical position of the knot points and the slopes to find the line which fits your data best. 

Such a model is known as a **Generalised Additive Model (GAM)**. Here you can find a comprehensive book on how these are designed and applied:
[Wood, S.N., 2017. Generalized additive models: an introduction with R. Chapman and Hall/CRC. https://www.taylorfrancis.com/books/9781498728348 ]

You can use the function **gam()** from the R package **mgcv** to fit such a model: 

In [ ]:
# load the library
library("mgcv")
# fit a gam model
x_values=seq(1880,2020,by=20)
my_gam = gam(Cya~s(Age, bs="cr", k=8), data=Cyano_rec, knots=list(Age=x_values), optimMethod="ML", sp=0)
# plot the data
plot(Cyano_rec$Age, Cyano_rec$Cya)
# plot the model
lines(Cyano_rec$Age,predict(my_gam,Cyano_rec))
# calculate the knots and plot them
y_values = predict(my_gam,data.frame(Age=x_values))
points(x_values,y_values,col="red",pch=15)

The parameters (y_values and slopes) are stored in a parameter vector $\mathbf{\beta}$. This was optimised to maximise the likelihood $l(\mathbf{\beta})$.

$$ \mathbf{y} = \beta_0 + s(\mathbf{x},\mathbf{\beta}) + \mathbf{\varepsilon} $$
$$ \mathbf{y} = \beta_0 + \mathbf{X} \cdot \mathbf{\beta} + \mathbf{\varepsilon} $$

The smoothness now depends on the number of knots we use. We can, however, ask the fitting function to produce a smoother spline. 

We want to punish if the second derivative (=curvature) gets too large, that is, we want to keep
$$ \int_{x} (s''(x,\mathbf{\beta}))^2 $$
small.

The smoothing parameter **sp** controls this.  It means that we will fit a restricted likelihood
$$ l_{restricted}(\mathbf{\beta}) = l(\mathbf{\beta}) - sp \cdot \mathbf{\beta} \cdot \mathbf{S} \cdot \mathbf{\beta} $$

Let's try different values:

In [ ]:
# fit three models
my_gam1 = gam(Cya~s(Age, bs="cr", k=8), data=Cyano_rec, knots=list(Age=x_values), optimMethod="ML", sp=1)
my_gam2 = gam(Cya~s(Age, bs="cr", k=8), data=Cyano_rec, knots=list(Age=x_values), optimMethod="ML", sp=2)
my_gam5 = gam(Cya~s(Age, bs="cr", k=8), data=Cyano_rec, knots=list(Age=x_values), optimMethod="ML", sp=5)
# plot them all
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_gam,Cyano_rec))
lines(Cyano_rec$Age,predict(my_gam1,Cyano_rec),col="blue")
lines(Cyano_rec$Age,predict(my_gam2,Cyano_rec),col="green")
lines(Cyano_rec$Age,predict(my_gam5,Cyano_rec),col="orange")

By default, the method will choose an appropriate value for sp automatically.

In [ ]:
# auto-select sp
my_gam_auto = gam(Cya~s(Age, bs="cr", k=8), data=Cyano_rec, knots=list(Age=x_values), optimMethod="ML")
# plot all
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_gam,Cyano_rec))
lines(Cyano_rec$Age,predict(my_gam1,Cyano_rec),col="blue")
lines(Cyano_rec$Age,predict(my_gam2,Cyano_rec),col="green")
lines(Cyano_rec$Age,predict(my_gam5,Cyano_rec),col="orange")
lines(Cyano_rec$Age,predict(my_gam_auto,Cyano_rec),col="red")
# show chosen value
my_gam_auto$sp

We may also want to take temporal autocorrelation of the data into account. Then we need to choose a **Generalised Additive Mixed Model (GAMM)**.

The fitting procedure is similar.

In [ ]:
my_gamm = gamm(Cya ~ s(Age, bs="cr", k = 8), data=Cyano_rec, 
               knots=list(Age=x_values), optimMethod="ML", 
               correlation = corCAR1(form = ~ Age))
# print the summary of the gamm
summary(my_gamm)

We see that the model has two components: 
* **lme** contains the local mixed-effects model
* **gam** contains the generalised additive model

The actual GAMM is a combination of the two. We can use the **gam** model part as before and plot the estimated trend:

In [ ]:
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age,predict(my_gam_auto,Cyano_rec),col="red")
lines(Cyano_rec$Age,predict(my_gamm$gam,Cyano_rec),col="violet")

The **lme** part will tell us about the temporal autocorrelation which was estimated.

In [ ]:
summary(my_gamm$lme)

Let's check the residuals for autocorrelation:

In [ ]:
acf(resid(my_gamm$lme,type="normalized"))

they don't show a significant autocorrelation any more, so we may say the cAR1 model describes the autocorrelation well enough.

The nonparametric fit also gives confidence intervals for the spline:

In [ ]:
# do a prediction with standard error
my_prediction = predict.gam(my_gamm$gam,se.fit=TRUE)
summary(my_prediction)

We see we have two entries: **fit** and **se.fit**.

In [ ]:
# plot data and model
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age, my_prediction$fit, col="violet")

# plot 95% confidence interval
lines(Cyano_rec$Age,my_prediction$fit + 1.96*my_prediction$se.fit,col="violet",lty=2)
lines(Cyano_rec$Age,my_prediction$fit - 1.96*my_prediction$se.fit,col="violet",lty=2)

Even for the trend, we can see where it is significantly different from zero. We take the following functions for granted:

[ https://www.fromthebottomoftheheap.net/2014/05/15/identifying-periods-of-change-with-gams/ ]

In [ ]:
################################################
## Functions for derivatives of GAM(M) models ##
################################################
Deriv <- function(mod, n = 200, eps = 1e-7, newdata, term) {
    if(inherits(mod, "gamm"))
        mod <- mod$gam
    m.terms <- attr(terms(mod), "term.labels")
    if(missing(newdata)) {
        newD <- sapply(model.frame(mod)[, m.terms, drop = FALSE],
                       function(x) seq(min(x), max(x), length = n))
        names(newD) <- m.terms
    } else {
        newD <- newdata
    }
    newDF <- data.frame(newD) ## needs to be a data frame for predict
    X0 <- predict(mod, newDF, type = "lpmatrix")
    newDF <- newDF + eps
    X1 <- predict(mod, newDF, type = "lpmatrix")
    Xp <- (X1 - X0) / eps
    Xp.r <- NROW(Xp)
    Xp.c <- NCOL(Xp)
    ## dims of bs
    bs.dims <- sapply(mod$smooth, "[[", "bs.dim") - 1
    ## number of smooth terms
    t.labs <- attr(mod$terms, "term.labels")
    ## match the term with the the terms in the model
    if(!missing(term)) {
        want <- grep(term, t.labs)
        if(!identical(length(want), length(term)))
            stop("One or more 'term's not found in model!")
        t.labs <- t.labs[want]
    }
    nt <- length(t.labs)
    ## list to hold the derivatives
    lD <- vector(mode = "list", length = nt)
    names(lD) <- t.labs
    for(i in seq_len(nt)) {
        Xi <- Xp * 0
        want <- grep(t.labs[i], colnames(X1))
        Xi[, want] <- Xp[, want]
        df <- Xi %*% coef(mod)
        df.sd <- rowSums(Xi %*% mod$Vp * Xi)^.5
        lD[[i]] <- list(deriv = df, se.deriv = df.sd)
    }
    class(lD) <- "Deriv"
    lD$gamModel <- mod
    lD$eps <- eps
    lD$eval <- newD - eps
    lD ##return
}
                
# return those points where the derivative is 95% significant                       
signif_deriv = function(my_gamm,term,other_terms=NULL) {
    my_deriv = Deriv(my_gamm)
    deriv_is_significant = abs(my_deriv[[term]]$deriv)>1.96*abs(my_deriv[[term]]$se.deriv)
    x_values = data.frame(V1 = my_deriv$eval)
    colnames(x_values)=term
    if (!is.null(other_terms)) {
        for (other_term in other_terms) {
            x_values[,other_term]=0
        }
    }
    y_values = predict.gam(my_gamm$gam,newdata=x_values,type="terms")[,paste0("s(",term,")")]
    y_values[!deriv_is_significant]=NA
    y_values = y_values + my_gamm$gam$coefficients[1]
    return(data.frame(x=x_values[,term],y=y_values))
}

Let us calculate the points where the derivative is significantly different from zero:

In [ ]:
# calculate these points
significant_derivative_points = signif_deriv(my_gamm,"Age")
# plot points and model estimate
plot(Cyano_rec$Age, Cyano_rec$Cya)
lines(Cyano_rec$Age, my_prediction$fit, col="violet")
lines(Cyano_rec$Age,my_prediction$fit + 1.96*my_prediction$se.fit,col="violet",lty=2)
lines(Cyano_rec$Age,my_prediction$fit - 1.96*my_prediction$se.fit,col="violet",lty=2)
# plot a thicker line (line width = 2) to show where the trend is significant
lines(significant_derivative_points,col="violet",lwd=2)

In [ ]:
?gamm

The model suggests two distinct increases of the cyanobacteria biomarker: 
* between 1880 and 1940
* between 1970 and now

But only in 1980-2015, the model has a 95% confidence that there was a systematic increasing trend.

## Example 2: Observed surface temperature at station BY31

The ICES database contains observation data for Landsort Deep SST from the 19th century on. Can we find out if there is a long-term trend?

Let us first load the data.

In [ ]:
icesdata = read.csv("../Data/sst_by31.csv",sep=";")
head(icesdata)
plot(icesdata)

Let's try to do a GAMM analysis as we did before.

In [ ]:
ices_gamm = gamm(temperature ~ s(decimalyear, bs="cr", k = 8) , data=icesdata, 
                 knots=list(decimalyear=x_values), optimMethod="ML")
plot(icesdata$decimalyear, icesdata$temperature)
lines(icesdata$decimalyear, predict(ices_gamm$gam), col="violet")

That looks weird: Do we really see a cooling here?

These data are irregular and have a pronounced seasonal cycle.

You may find a **seasonal observation bias**: 
Assume you find a long-term trend - is that 
* because it is actually getting warmer, or
* because you measured more often in summer?

Let's add a column **season** to the data frame.

In [ ]:
# get the year
icesdata$year = floor(icesdata$decimalyear)
# subtract it from decimalyear to get the season
icesdata$season = icesdata$decimalyear - icesdata$year

head(icesdata)

We can now fit a statistical model like this:
$$ temperature = \beta_0 + s_1(decimalyear) + s_2(season) + \varepsilon $$

which explicitly contains seasonality.

This idea follows this online tutorial:
[ https://www.fromthebottomoftheheap.net/2014/05/09/modelling-seasonal-data-with-gam/ ]

In [ ]:
ices_gamm = gamm(temperature ~ s(decimalyear, bs="cr", k = 8) + s(season, bs="cc",k=12), data=icesdata, 
                 knots=list(decimalyear=x_values, season=c(0,1)), optimMethod="ML", 
                 correlation = corCAR1(form = ~ decimalyear|year))
plot(ices_gamm$gam)

Now we want to plot the temperature trend together with the data. To do so, we have to make a prediction term by term:

In [ ]:
ices_gamm$gam$coefficients

In [ ]:
# get the intercept - it is the first of the coefficients
intercept = ices_gamm$gam$coefficients[1]
intercept

# predict the terms one by one
my_prediction = predict(ices_gamm$gam,se.fit=TRUE,type="terms")
print(my_prediction)

In [ ]:
# get the temperature from the first columm
pred_temperature = intercept + my_prediction$fit[,1]

# get minimum and maximum temperature
pred_temperature_min = intercept + my_prediction$fit[,1] - 1.96*my_prediction$se.fit[,1]
pred_temperature_max = intercept + my_prediction$fit[,1] + 1.96*my_prediction$se.fit[,1]

# do the plot
plot(icesdata$decimalyear, icesdata$temperature)
lines(icesdata$decimalyear, pred_temperature, col="red")
lines(icesdata$decimalyear, pred_temperature_min, lty=2, col="red")
lines(icesdata$decimalyear, pred_temperature_max, lty=2, col="red")


Where is the trend significant?

In [ ]:
significant_derivative_points = signif_deriv(ices_gamm,term="decimalyear",other_terms = c("season"))
plot(icesdata$decimalyear, icesdata$temperature)
lines(icesdata$decimalyear, pred_temperature, col="red")
lines(icesdata$decimalyear, pred_temperature_min, lty=2, col="red")
lines(icesdata$decimalyear, pred_temperature_max, lty=2, col="red")
lines(significant_derivative_points, lwd=2, col="blue")